In [1]:
from torch_geometric.datasets import MoleculeNet
import seaborn as sns
import numpy as np
import pandas as pd
from IPython.display import clear_output
from scipy.spatial import cKDTree
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import torch.nn as nn
import torch_geometric.nn as geom_nn
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
import pickle
import math

import torch
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn.dense import DenseGCNConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Matplotlib created a temporary cache directory at /localscratch-ssd/289625/matplotlib-yoep0oz9 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [18]:
### Load the list from the pickle file ###
### Train Data ###
with open('train_data_np_scaled.pkl', 'rb') as f:
    train_data_np_scaled = pickle.load(f)

### Test Data ###
with open('test_data_np_scaled.pkl', 'rb') as f:
    test_data_np_scaled = pickle.load(f)

In [19]:
train_data_np_scaled[0]

array([[0.34407167, 0.34918513, 0.56854133, 0.63448512, 0.33022027,
        0.40517533, 0.62870267, 0.53271002, 0.38819273, 0.28237867,
        0.525932  , 0.6717093 , 0.32078433, 0.26959267, 0.62823667,
        0.72224701, 0.400144  , 0.347111  , 0.46699   , 0.64623932,
        0.27414187, 0.35712667, 0.66621133, 0.6190243 , 0.243908  ,
        0.38105533, 0.49648533, 0.56017708, 0.37557587, 0.26455933,
        0.69015347, 0.58668433, 0.36889   , 0.22820533, 0.480584  ,
        0.59320475, 0.24428867, 0.25186333, 0.629588  , 0.59093708,
        0.19227533, 0.29914933, 0.55064333, 0.56045067, 0.36573533,
        0.3153772 , 0.410002  , 0.53469994, 0.22763533, 0.256208  ,
        0.49946133, 0.62557304, 0.450466  , 0.47587867, 0.54980467,
        0.49193433, 0.19378733, 0.31341667, 0.63136913, 0.55318804,
        0.36573533, 0.3153772 , 0.74333533, 0.53469994, 0.        ,
        0.        , 0.        , 0.63448512, 0.18889798],
       [0.33643627, 0.35437233, 0.572982  , 0.59721892, 0.3

In [ ]:
### Load Data (Ze Time series data) ###
location = "simple_connections_data/random_split/"
experiment = "rho100_10percent_Re100/"

train_input = np.load(location+experiment+"train_inputs.npy")
test_input = np.load(location+experiment+"test_inputs.npy")

train_inputs_global = torch.tensor(np.load(location+experiment+"train_input_scalar.npy"))
test_inputs_global = torch.tensor(np.load(location+experiment+"test_input_scalar.npy"))

train_output = np.load(location+experiment+"train_output.npy")
test_output = np.load(location+experiment+"test_output.npy")

### edge index for basic connections ###
edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                           [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]])

train_combined = list()
test_combined = list()

### Stacking up train data ###
for i in range(len(train_input)):

    ### setting inputs ###
    x = torch.tensor(train_input[i]).float().clone().detach()
    
    ### adding drag force as y ###
    y = torch.tensor(train_output[i]).float().clone().detach()
    
    # all_data_graph_struct.append(Data(x=x , edge_index=torch.tensor(mirror_edge_index(edge_index)) , y=y))
    train_combined.append(Data(x=x.clone().detach() , edge_index=edge_index.clone().detach() , y=y[:,None].clone().detach()))

### Stacking up test data ###
for i in range(len(test_input)):

    ### setting inputs ###
    x = torch.tensor(test_input[i]).float().clone().detach()
    
    ### adding drag force as y ###
    y = torch.tensor(test_output[i]).float().clone().detach()
    
    # all_data_graph_struct.append(Data(x=x , edge_index=torch.tensor(mirror_edge_index(edge_index)) , y=y))
    test_combined.append(Data(x=x.clone().detach() , edge_index=edge_index.clone().detach() , y=y[:,None].clone().detach()))

# Graph Attention Network

In [10]:
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, global_add_pool as gap
from torch.nn import Linear

class GNN_with_attention(torch.nn.Module):
    def __init__(self, num_gcn_layers=2, num_fcnn_layers=2, gcn_embedding_size=64, fcnn_embedding_size=64,
                 num_nodes=16, num_features=4, dropout_prob=0.5, heads=2):
        super(GNN_with_attention, self).__init__()
        torch.manual_seed(42)

        self.num_gcn_layers = num_gcn_layers
        self.num_fcnn_layers = num_fcnn_layers
        self.dropout_prob = dropout_prob

        # GATv2Conv layers
        self.initial_conv = GATv2Conv(num_features, gcn_embedding_size, heads=heads)
        self.convs = torch.nn.ModuleList()
        
        for _ in range(num_gcn_layers - 1):
            self.convs.append(GATv2Conv(gcn_embedding_size * heads, gcn_embedding_size, heads=heads))

        # FCNN layers
        self.fcnn_layers = torch.nn.ModuleList()
        if num_fcnn_layers > 0:
            self.fcnn_layers.append(Linear(gcn_embedding_size * heads + 4, fcnn_embedding_size))
            for _ in range(num_fcnn_layers - 2):
                self.fcnn_layers.append(Linear(fcnn_embedding_size, fcnn_embedding_size))
            self.fcnn_layers.append(Linear(fcnn_embedding_size, 1))

        # Dropout layer
        self.dropout = torch.nn.Dropout(p=self.dropout_prob)

    def forward(self, x, edge_index, x_scalar, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.relu(hidden)
        
        # Other Conv layers with dropout at every other layer
        for i, conv in enumerate(self.convs):
            hidden = conv(hidden, edge_index)
            hidden = F.relu(hidden)
            if i % 2 == 1:
                hidden = self.dropout(hidden)

        # Graph aggregation
        hidden = gap(hidden, batch_index)   
        hidden = torch.cat((hidden, x_scalar), axis=1)

        # FCNN layers
#         for i, layer in enumerate(self.fcnn_layers):
#             hidden = layer(hidden)
#             if i < len(self.fcnn_layers) - 1:  # Apply ReLU to all but the last layer
#                 hidden = F.relu(hidden)

        return hidden

model = GNN_with_attention(num_gcn_layers=3, num_fcnn_layers=3, gcn_embedding_size=85, fcnn_embedding_size=64,
                 num_nodes=16, num_features=4, dropout_prob=0.2, heads=2)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

### example implementation ###

### edge index for basic connections ###
edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                           [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]])

# x, edge_index, x_scalar, batch_index
test_output = model(torch.randn(16,4),edge_index,
                   torch.randn(1,4),torch.zeros(16).int().to(torch.int64))
test_output.shape

GNN_with_attention(
  (initial_conv): GATv2Conv(4, 85, heads=2)
  (convs): ModuleList(
    (0): GATv2Conv(170, 85, heads=2)
    (1): GATv2Conv(170, 85, heads=2)
  )
  (fcnn_layers): ModuleList(
    (0): Linear(in_features=174, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (dropout): Dropout(p=0.2, inplace=False)
)
Number of parameters:  134425


torch.Size([1, 174])

In [17]:
i=0
temp = torch.stack([model(torch.randn(16,4)*i,edge_index,
                   torch.randn(1,4),torch.zeros(16).int().to(torch.int64)) for i in range(4)]).transpose(1,0)
temp.shape

torch.Size([1, 4, 174])

# Transformer

In [3]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TimeSeriesTransformerEncoder(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, dim_feedforward=2048, max_len=5000):
        super(TimeSeriesTransformerEncoder, self).__init__()
        self.d_model = d_model

        self.input_projection = nn.Linear(input_dim, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.output_projection = nn.Linear(d_model, 1)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x = self.input_projection(x)  # (batch_size, seq_len, d_model)
        x = x.transpose(0, 1)  # (seq_len, batch_size, d_model)
        x = self.positional_encoding(x)  # (seq_len, batch_size, d_model)
        x = self.transformer_encoder(x)  # (seq_len, batch_size, d_model)
        x = x.transpose(0, 1)  # (batch_size, seq_len, d_model)
        x = x.transpose(1, 2)  # (batch_size, d_model, seq_len)
        x = self.pooling(x)  # (batch_size, d_model, 1)
        x = x.squeeze(2)  # (batch_size, d_model)
        x = self.output_projection(x)  # (batch_size, 1)
        return x

# Example usage:
# input_dim = 10  # Number of features in the time series data
# d_model = 64    # Dimensionality of the embeddings
# nhead = 8       # Number of attention heads
# num_layers = 6  # Number of transformer encoder layers

model = TimeSeriesTransformerEncoder(input_dim=174, d_model=66, nhead=3, num_layers=2)
input_tensor = torch.rand(32, 5, 174)  # (batch_size, seq_len, input_dim)
output = model(input_tensor)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))
print(output.shape)  # (batch_size, 1)

Number of parameters:  592421
torch.Size([32, 1])


# Combined_model

In [5]:
class GNN_Transformer(torch.nn.Module):
    
    def __init__(self, num_gcn_layers=2, num_fcnn_layers=2, gcn_embedding_size=128, fcnn_embedding_size=64,
                 num_nodes=16, num_features=4, dropout_prob=0.2, heads=2, ### GAT parameters
                 input_dim=132, d_model=124, nhead=4, num_layers=2, dim_feedforward=64, max_len=5): ### Transformer parameters
        
        super(GNN_Transformer, self).__init__()
        self.GNN = GNN_with_attention(num_gcn_layers,num_fcnn_layers,gcn_embedding_size,fcnn_embedding_size,
                                      num_nodes,num_features,dropout_prob,heads)
        self.transformer = TimeSeriesTransformerEncoder(input_dim,d_model,nhead,
                                                        num_layers,dim_feedforward,max_len)
        
    def forward(self,x, edge_index, x_scalar, batch_index):
        
        gnn_embedding = self.GNN(x, edge_index, x_scalar, batch_index)
        print("gnn_embedding",gnn_embedding.shape)
        exit()
        prediction = self.transformer(gnn_embedding)
        
        return prediction
    
gnn_transformer = GNN_Transformer(num_gcn_layers=3, num_fcnn_layers=3, gcn_embedding_size=85, fcnn_embedding_size=64,
                                  num_nodes=16, num_features=4, dropout_prob=0.2, heads=2,
                                  input_dim=174, d_model=66, nhead=3, num_layers=2)

print("Number of parameters: ", sum(p.numel() for p in gnn_transformer.parameters()))

# output = gnn_transformer(torch.randn(16,4),edge_index,
#                    torch.randn(1,4),torch.zeros(16).int().to(torch.int64))
# output.shape

Number of parameters:  199102


In [ ]:
# Wrap data in a data loader
# NUM_GRAPHS_PER_BATCH = 64
# N = 36000

# train_loader = DataLoader(list(zip(train_combined[0:N],train_inputs_global[0:N])), 
#                     batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

# test_loader = DataLoader(list(zip(test_combined,test_inputs_global)), 
#                     batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

Run Training

In [ ]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.00025)  

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### lr scheduler ###
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

epoch_loss_train = list()
epoch_loss_val = list()
lr_list = list()

# model.load_state_dict(torch.load("model_general"))
# model = model.to(device)

save_loc = ""

for epoch in range(0,100):
    print(f'Starting Epoch {epoch+1}')

    current_loss = 0.0
    loss_train = list()
    loss_val = list()
    
    for batch,inputs_global in train_loader:

        batch.to(device)
        inputs_global = inputs_global.float().cuda() 

        optimizer.zero_grad()

        pred = model( batch.x.float() , batch.edge_index, inputs_global, batch.batch)
        
        loss = loss_fn(pred, batch.y)
        loss.backward()  
        
        # Update using the gradients
        optimizer.step()   

        current_loss += loss.item()
        loss_train.append(loss.item())
        
    for batch,inputs_global in test_loader:

        batch.to(device)
        inputs_global = inputs_global.float().cuda() 
        
        pred = model( batch.x.float() , batch.edge_index,inputs_global, batch.batch)
        
        loss = loss_fn(pred,batch.y)
        loss_val.append(loss.item())
        
    print(f'Epoch {epoch+1} finished with training loss = '+str(np.array(loss_train).mean()))
    print(f'testing loss = '+str(np.array(loss_val).mean()) + '\n' )

    epoch_loss_train.append(np.array(loss_train).mean())
    epoch_loss_val.append(np.array(loss_val).mean())

    ### applying lr scheduling ###
    lr_list.append(optimizer.param_groups[0]['lr'])
    scheduler.step(epoch_loss_train[-1])

    if epoch%1==0:
        
        torch.save(model.state_dict(), save_loc+'model_'+str(epoch))

    np.save(save_loc+"epoch_loss_train",epoch_loss_train)
    np.save(save_loc+"epoch_loss_val",epoch_loss_val)

print("Training has completed")

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(epoch_loss_train)
plt.semilogy(epoch_loss_val)
plt.semilogy(lr_list)
plt.savefig(location+experiment+"train_val_loss_vs_epochs")
print(epoch_loss_train[-1],epoch_loss_val[-1])